<a href="https://colab.research.google.com/github/alim98/MPI/blob/main/Vis_vit_2d_connected_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download

In [1]:
!wget -O downloaded_file.zip "https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000"
!pip -q install umap-learn
!unzip -q downloaded_file.zip
!wget -O ViT_all_features_merged.csv "https://drive.usercontent.google.com/download?id=1rOpINPMO5yqXycFFbJsqURRECwMiHUmP&export=download"


--2025-01-10 13:17:49--  https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.210.132, 2607:f8b0:400c:c0f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.210.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1264688649 (1.2G) [application/octet-stream]
Saving to: ‘downloaded_file.zip’

downloaded_file.zip 100%[===================>]   1.18G  57.6MB/s    in 18s     

2025-01-10 13:18:07 (67.5 MB/s) - ‘downloaded_file.zip’ saved [1264688649/1264688649]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.4 MB/s eta 0:00:00
--2025-01-10 13:19:33--  https://drive.usercontent.google.com/download?id=1rOpINPMO5y

# Just Vit direct Umap

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import umap
import plotly.graph_objects as go

def process_features(df):
    feature_cols = [c for c in df.columns if c.startswith('feat_')]
    features = df[feature_cols].values

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    return features_scaled

def reduce_to_umap(features_scaled):
    umap_direct = umap.UMAP(n_components=2, random_state=42)
    umap_direct_result = umap_direct.fit_transform(features_scaled)

    return umap_direct_result

def create_vit_umap_plot():
    # Define consistent colors for each bbox
    color_mapping = {
        'bbox1': '#FF0000',  # Red
        'bbox2': '#00FFFF',  # Cyan
        'bbox3': '#FFA500',  # Orange
        'bbox4': '#800080',  # Purple
        'bbox5': '#808080',  # Gray
        'bbox6': '#0000FF',  # Blue
        'bbox7': '#000000'   # Black
    }

    file_path = 'ViT_all_features_merged.csv'
    df = pd.read_csv(file_path)
    features_scaled = process_features(df)
    umap_direct_result = reduce_to_umap(features_scaled)

    fig = go.Figure()

    for bbox in sorted(df['bbox_name'].unique()):
        mask = df['bbox_name'] == bbox

        fig.add_trace(
            go.Scatter(
                x=umap_direct_result[mask, 0],
                y=umap_direct_result[mask, 1],
                mode='markers',
                name=bbox,
                marker=dict(
                    size=5,
                    color=color_mapping.get(bbox, '#000000'),  # Default to black if bbox not in mapping
                    opacity=0.7
                ),
                text=df[mask]['Var1'],
                hovertemplate=(
                    'Label: %{text}<br>'
                    'x: %{x:.2f}<br>'
                    'y: %{y:.2f}<br>'
                    'bbox: ' + bbox + '<br>'
                    '<extra></extra>'
                )
            )
        )

    fig.update_layout(
        height=600,
        width=800,
        title_text="ViT - Direct UMAP",
        showlegend=True,
        xaxis_title="UMAP 1",
        yaxis_title="UMAP 2"
    )

    return fig

fig = create_vit_umap_plot()
fig.show()

fig.write_html("vit_direct_umap.html")
print("Visualization has been saved as 'vit_direct_umap.html'")


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Visualization has been saved as 'vit_direct_umap.html'


# Visualizing

In [19]:

import os
import numpy as np
import pandas as pd
from PIL import Image
import base64
import io
import imageio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import umap
import glob
color_mapping = {
    'bbox1': '#FF0000',  # Red
    'bbox2': '#00FFFF',  # Cyan
    'bbox3': '#FFA500',  # Orange
    'bbox4': '#800080',  # Purple
    'bbox5': '#808080',  # Gray
    'bbox6': '#0000FF',  # Blue
    'bbox7': '#000000'   # Black
}

def process_features(df):
    feature_cols = [c for c in df.columns if c.startswith('feat_')]
    features = df[feature_cols].values
    features_scaled = StandardScaler().fit_transform(features)
    return features_scaled

file_path = 'ViT_all_features_merged.csv'
df = pd.read_csv(file_path)

if 'id' not in df.columns:
    df['id'] = df.index
def reduce_to_umap(features_scaled):
    umap_direct = umap.UMAP(n_components=2, random_state=42)
    return umap_direct.fit_transform(features_scaled)

features_scaled = process_features(df)
umap_result = reduce_to_umap(features_scaled)

df['umap_1'] = umap_result[:, 0]
df['umap_2'] = umap_result[:, 1]

sampled_points = df.sample(20, random_state=42).sort_values('id')
class VolumeProcessor:
    def __init__(self, raw_base_dir, seg_base_dir, bbox_name, excel_file):
        self.raw_base_dir = raw_base_dir
        self.seg_base_dir = seg_base_dir
        self.bbox_name = bbox_name
        self.excel_file = excel_file
        self.raw_vol = None
        self.seg_vol = None
        self.synapse_data = None

    def load_volume_data(self, max_slices=None):
        raw_dir = os.path.join(self.raw_base_dir, self.bbox_name)
        seg_dir = os.path.join(self.seg_base_dir, self.bbox_name)

        raw_files = sorted(glob.glob(os.path.join(raw_dir, 'slice_*.tif')))
        seg_files = sorted(glob.glob(os.path.join(seg_dir, 'slice_*.tif')))

        if max_slices:
            raw_files = raw_files[:max_slices]
            seg_files = seg_files[:max_slices]

        assert len(raw_files) == len(seg_files), "Number of raw and segmentation files do not match."

        self.raw_vol = np.stack([imageio.imread(f) for f in raw_files], axis=0)
        self.seg_vol = np.stack([imageio.imread(f).astype(np.uint32) for f in seg_files], axis=0)

    def load_synapse_data(self):
        self.synapse_data = pd.read_excel(self.excel_file)

    def create_segment_masks(self, side1_coord, side2_coord):
        x1, y1, z1 = map(int, side1_coord)
        x2, y2, z2 = map(int, side2_coord)

        seg_id_1 = self.seg_vol[z1, y1, x1]
        seg_id_2 = self.seg_vol[z2, y2, x2]

        mask_1 = (self.seg_vol == seg_id_1) if seg_id_1 != 0 else np.zeros_like(self.seg_vol, dtype=bool)
        mask_2 = (self.seg_vol == seg_id_2) if seg_id_2 != 0 else np.zeros_like(self.seg_vol, dtype=bool)
        return mask_1, mask_2

    def process_synapses(self, start_idx, end_idx, subvolume_size=80):
        half_size = subvolume_size // 2

        for idx in range(start_idx, end_idx):
            syn_info = self.synapse_data.iloc[idx]

            central_coord = (
                int(syn_info['central_coord_1']),
                int(syn_info['central_coord_2']),
                int(syn_info['central_coord_3'])
            )

            side1_coord = (
                int(syn_info['side_1_coord_1']),
                int(syn_info['side_1_coord_2']),
                int(syn_info['side_1_coord_3'])
            )

            side2_coord = (
                int(syn_info['side_2_coord_1']),
                int(syn_info['side_2_coord_2']),
                int(syn_info['side_2_coord_3'])
            )

            mask_1_full, mask_2_full = self.create_segment_masks(side1_coord, side2_coord)

            cx, cy, cz = central_coord

            x_start, x_end = max(cx - half_size, 0), min(cx + half_size, self.raw_vol.shape[2])
            y_start, y_end = max(cy - half_size, 0), min(cy + half_size, self.raw_vol.shape[1])
            z_start, z_end = max(cz - half_size, 0), min(cz + half_size, self.raw_vol.shape[0])

            sub_raw = self.raw_vol[z_start:z_end, y_start:y_end, x_start:x_end]
            sub_mask_1 = mask_1_full[z_start:z_end, y_start:y_end, x_start:x_end]
            sub_mask_2 = mask_2_full[z_start:z_end, y_start:y_end, x_start:x_end]

            self._generate_gif(sub_raw, sub_mask_1, sub_mask_2, syn_info, z_start)

    def _generate_gif(self, sub_raw, sub_mask_1, sub_mask_2, syn_info, z_start):
        fig, ax = plt.subplots(figsize=(5, 5))
        im = ax.imshow(sub_raw[0], cmap='gray', interpolation='nearest')
        ax.axis('off')

        legend_patches = [
            Patch(facecolor='red', edgecolor='red', label='Side 1'),
            Patch(facecolor='blue', edgecolor='blue', label='Side 2')
        ]
        ax.legend(handles=legend_patches, loc='upper right', frameon=True)

        def update(frame):
            overlay = np.stack([sub_raw[frame]] * 3, axis=-1).astype(float)

            overlay[sub_mask_1[frame]] = overlay[sub_mask_1[frame]] * 0.7 + np.array([255, 0, 0]) * 0.3
            overlay[sub_mask_2[frame]] = overlay[sub_mask_2[frame]] * 0.7 + np.array([0, 0, 255]) * 0.3

            overlay = np.clip(overlay, 0, 255).astype(np.uint8)
            ax.set_title(f"{syn_info['Var1']} - Z: {z_start + frame}")
            im.set_data(overlay)
            return [im]

        ani = animation.FuncAnimation(fig, update, frames=sub_raw.shape[0], interval=100, blit=True)

        gif_filename = f"{syn_info['Var1']}.gif"
        ani.save(gif_filename, writer='pillow', fps=10)
        plt.close(fig)
        print(f"Saved: {gif_filename}")

# Helper function to convert NumPy array to base64-encoded string
def numpy_to_base64(img_array):
    pil_image = Image.fromarray(img_array.astype('uint8'))  # Convert to PIL Image
    buffer = io.BytesIO()
    pil_image.save(buffer, format="PNG")  # Save as PNG to a buffer
    buffer.seek(0)
    img_base64 = base64.b64encode(buffer.read()).decode('utf-8')  # Encode to base64
    return f"data:image/png;base64,{img_base64}"

# Helper function to overlay side masks on a patch
def overlay_side_masks(raw_patch, mask_side1, mask_side2):
    """
    Overlays side1 (blue) and side2 (red) masks onto a raw patch.

    Args:
        raw_patch: NumPy array of the raw patch (grayscale).
        mask_side1: Boolean mask for side1.
        mask_side2: Boolean mask for side2.

    Returns:
        NumPy array of the patch with side overlays.
    """
    # Convert grayscale patch to RGB
    patch_rgb = np.stack([raw_patch] * 3, axis=-1).astype(float)

    # Ensure no overlap between side1 and side2
    mask_side1 = mask_side1 & ~mask_side2
    mask_side2 = mask_side2 & ~mask_side1

    # Apply blue overlay for side1 (50% transparency)
    patch_rgb[mask_side1] = patch_rgb[mask_side1] * 0.5 + np.array([0, 0, 255]) * 0.5

    # Apply red overlay for side2 (50% transparency)
    patch_rgb[mask_side2] = patch_rgb[mask_side2] * 0.5 + np.array([255, 0, 0]) * 0.5

    # Clip values to valid range and return
    return np.clip(patch_rgb, 0, 255).astype(np.uint8)

# Function to generate and save GIFs for each sample
def generate_gif_for_sample(raw_vol, mask_side1_full, mask_side2_full, central_coord, var1, output_dir, half_size=40):
    """
    Generates a GIF for a given sample with side overlays.

    Args:
        raw_vol: Full raw volume data.
        mask_side1_full: Full mask for side1.
        mask_side2_full: Full mask for side2.
        central_coord: Central coordinate of the sample.
        var1: Variable name for the sample (used for the GIF filename).
        output_dir: Directory to save the GIFs.
        half_size: Half the size of the extracted subvolume.
    """
    cx, cy, cz = central_coord
    x_start, x_end = max(cx - half_size, 0), min(cx + half_size, raw_vol.shape[2])
    y_start, y_end = max(cy - half_size, 0), min(cy + half_size, raw_vol.shape[1])
    z_start, z_end = max(cz - half_size, 0), min(cz + half_size, raw_vol.shape[0])

    sub_raw = raw_vol[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_1 = mask_side1_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_2 = mask_side2_full[z_start:z_end, y_start:y_end, x_start:x_end]

    # Create a directory for GIFs if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    gif_path = os.path.join(output_dir, f"{var1}.gif")

    # Generate frames for the GIF
    frames = []
    for z in range(sub_raw.shape[0]):
        overlay = overlay_side_masks(sub_raw[z], sub_mask_1[z], sub_mask_2[z])
        frames.append(overlay)

    # Save the GIF
    imageio.mimsave(gif_path, frames, fps=5)
    print(f"Saved GIF: {gif_path}")


def overlay_side_masks(raw_patch, mask_side1, mask_side2):
    """
    Overlays side1 (blue) and side2 (red) masks onto a raw patch.

    Args:
        raw_patch: NumPy array of the raw patch (grayscale).
        mask_side1: Boolean mask for side1.
        mask_side2: Boolean mask for side2.

    Returns:
        NumPy array of the patch with side overlays.
    """
    # Convert grayscale patch to RGB
    patch_rgb = np.stack([raw_patch] * 3, axis=-1).astype(float)

    # Apply blue overlay for side1
    patch_rgb[mask_side1] = patch_rgb[mask_side1] * 0.7 + np.array([0, 0, 255]) * 0.3

    # Apply red overlay for side2
    patch_rgb[mask_side2] = patch_rgb[mask_side2] * 0.7 + np.array([255, 0, 0]) * 0.3

    # Clip values to valid range and return
    return np.clip(patch_rgb, 0, 255).astype(np.uint8)

# Initialize VolumeProcessor
processor = VolumeProcessor(raw_base_dir='/content/raw',
                             seg_base_dir='/content/seg',
                             bbox_name='bbox1',
                             excel_file='bbox1.xlsx')
processor.load_volume_data()
processor.load_synapse_data()

# Update patches with side overlays
updated_patches = []
for _, row in sampled_points.iterrows():
    central_coord = (int(row['central_coord_1']), int(row['central_coord_2']), int(row['central_coord_3']))
    side1_coord = (int(row['side_1_coord_1']), int(row['side_1_coord_2']), int(row['side_1_coord_3']))
    side2_coord = (int(row['side_2_coord_1']), int(row['side_2_coord_2']), int(row['side_2_coord_3']))

    # Create masks for side1 and side2
    mask_side1_full, mask_side2_full = processor.create_segment_masks(side1_coord, side2_coord)

    # Extract subvolumes for raw and masks
    half_size = 40
    x, y, z = central_coord
    x_start, x_end = max(x - half_size, 0), min(x + half_size, processor.raw_vol.shape[2])
    y_start, y_end = max(y - half_size, 0), min(y + half_size, processor.raw_vol.shape[1])
    z_start, z_end = max(z - half_size, 0), min(z + half_size, processor.raw_vol.shape[0])

    raw_patch = processor.raw_vol[z, y_start:y_end, x_start:x_end]
    mask_side1_patch = mask_side1_full[z, y_start:y_end, x_start:x_end]
    mask_side2_patch = mask_side2_full[z, y_start:y_end, x_start:x_end]

    # Overlay side masks onto the patch
    patch_with_sides = overlay_side_masks(raw_patch, mask_side1_patch, mask_side2_patch)
    updated_patches.append(patch_with_sides)

# Create patch grid plot with updated patches
grid_rows, grid_cols = 4, 5
patch_fig = make_subplots(
    rows=grid_rows,
    cols=grid_cols,
    subplot_titles=[f"ID: {id_}" for id_ in sampled_points['id']],
    vertical_spacing=0.05,
    horizontal_spacing=0.05
)

# Add patches with side overlays to the grid
for idx, patch in enumerate(updated_patches):
    row, col = divmod(idx, grid_cols)
    patch_base64 = numpy_to_base64(patch)

    patch_fig.add_trace(
        go.Image(source=patch_base64),
        row=row + 1,
        col=col + 1
    )

df['color'] = df['bbox_name'].map(color_mapping)
df['size'] = 5  # Default size for all points

# Update size for sampled points to make them bigger
sampled_points['color'] = sampled_points['bbox_name'].map(color_mapping)
sampled_points['size'] = 10  # Bigger size for sampled points

# Create UMAP scatter plot
scatter_fig = go.Figure()

# Add all points as a background
scatter_fig.add_trace(go.Scatter(
    x=df['umap_1'],
    y=df['umap_2'],
    mode='markers',
    marker=dict(color=df['color'], size=df['size'], opacity=0.7),
    name="All Points",
    legendgroup="All Points",  # Group for the legend
    showlegend=False,  # Hide all points from the legend
    hoverinfo='skip'  # Skip hover info for background points
))

# Add sampled points (with bigger size)
scatter_fig.add_trace(go.Scatter(
    x=sampled_points['umap_1'],
    y=sampled_points['umap_2'],
    mode='markers+text',
    marker=dict(color=sampled_points['color'], size=sampled_points['size']),
    text=[f"ID: {id_}" for id_ in sampled_points['id']],
    textposition='top center',
    name="Sampled Points",
    legendgroup="Sampled Points",  # Group for the legend
    showlegend=False,  # Hide sampled points from the legend
    hoverinfo='text',
    hovertext=[f"ID: {id_}, BBox: {bbox}" for id_, bbox in zip(sampled_points['id'], sampled_points['bbox_name'])]
))

# Add custom legend entries for bbox colors
for bbox, color in color_mapping.items():
    scatter_fig.add_trace(go.Scatter(
        x=[None],  # Dummy entry
        y=[None],
        mode='markers',
        marker=dict(color=color, size=6),
        name=bbox,  # Name of the bbox
        legendgroup=bbox,  # Group for the legend
    ))

# Customize layout
scatter_fig.update_layout(
    title="UMAP Scatter Plot with Color-Coded Legends",
    xaxis_title="UMAP 1",
    yaxis_title="UMAP 2",
    height=1500,
    width=1300,
    legend_title="Bounding Boxes",
)

# Show scatter plot
scatter_fig.show()

# UMAP Scatter Plot
scatter_fig = go.Figure()

# Create patch grid plot
grid_rows, grid_cols = 4, 5
patch_fig = make_subplots(
    rows=grid_rows,
    cols=grid_cols,
    subplot_titles=[
        f"ID: {row['id']}<br>BBox: {row['bbox_name']}<br>Var1: {row['Var1']}<br>Slice: {row['central_coord_3']}"
        for _, row in sampled_points.iterrows()
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.1
)

# Add patches with side overlays to the grid
for idx, patch in enumerate(updated_patches):
    row, col = divmod(idx, grid_cols)
    patch_base64 = numpy_to_base64(patch)

    patch_fig.add_trace(
        go.Image(source=patch_base64),
        row=row + 1,
        col=col + 1
    )

# Update layout
patch_fig.update_layout(
    title="",
    height=1200,
    width=1500,
    showlegend=False
)

# Reduce font size of subplot titles
for annotation in patch_fig['layout']['annotations']:
    annotation['font'] = dict(size=13)  # Set font size to 13 for clarity

# Show patch grid
patch_fig.show()

# Generate GIFs for each selected sample
gif_output_dir = "gifs"  # Directory to save GIFs
for _, row in sampled_points.iterrows():
    central_coord = (int(row['central_coord_1']), int(row['central_coord_2']), int(row['central_coord_3']))
    side1_coord = (int(row['side_1_coord_1']), int(row['side_1_coord_2']), int(row['side_1_coord_3']))
    side2_coord = (int(row['side_2_coord_1']), int(row['side_2_coord_2']), int(row['side_2_coord_3']))

    # Create masks for side1 and side2
    mask_side1_full, mask_side2_full = processor.create_segment_masks(side1_coord, side2_coord)

    # Generate GIF for the current sample
    generate_gif_for_sample(processor.raw_vol, mask_side1_full, mask_side2_full,
                            central_coord,row['id'], gif_output_dir)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

<ipython-input-19-3920e724b2da>:69: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

<ipython-input-19-3920e724b2da>:70: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



Saved GIF: gifs/9.gif
Saved GIF: gifs/30.gif
Saved GIF: gifs/55.gif
Saved GIF: gifs/63.gif
Saved GIF: gifs/68.gif
Saved GIF: gifs/70.gif
Saved GIF: gifs/73.gif
Saved GIF: gifs/79.gif
Saved GIF: gifs/84.gif
Saved GIF: gifs/124.gif
Saved GIF: gifs/131.gif
Saved GIF: gifs/167.gif
Saved GIF: gifs/185.gif
Saved GIF: gifs/247.gif
Saved GIF: gifs/284.gif
Saved GIF: gifs/317.gif
Saved GIF: gifs/397.gif
Saved GIF: gifs/473.gif
Saved GIF: gifs/486.gif
Saved GIF: gifs/494.gif


In [20]:

import plotly.io as pio


pio.write_html(patch_fig, file='patch_fig.html')
pio.write_html(scatter_fig, file='scatter_fig.html')

from google.colab import files
files.download('patch_fig.html')
files.download('scatter_fig.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:

!zip -r /content/gifs.zip /content/gifs


updating: content/gifs/ (stored 0%)
updating: content/gifs/486.gif (deflated 11%)
updating: content/gifs/79.gif (deflated 10%)
updating: content/gifs/55.gif (deflated 10%)
updating: content/gifs/185.gif (deflated 11%)
updating: content/gifs/68.gif (deflated 10%)
updating: content/gifs/73.gif (deflated 11%)
updating: content/gifs/84.gif (deflated 12%)
updating: content/gifs/131.gif (deflated 12%)
updating: content/gifs/284.gif (deflated 10%)
updating: content/gifs/63.gif (deflated 14%)
updating: content/gifs/30.gif (deflated 10%)
updating: content/gifs/9.gif (deflated 12%)
updating: content/gifs/317.gif (deflated 11%)
updating: content/gifs/473.gif (deflated 12%)
updating: content/gifs/397.gif (deflated 13%)
updating: content/gifs/167.gif (deflated 12%)
updating: content/gifs/124.gif (deflated 12%)
updating: content/gifs/494.gif (deflated 12%)
updating: content/gifs/247.gif (deflated 12%)
updating: content/gifs/70.gif (deflated 13%)
